In [1]:
from IPython.display import display
import pandas as pd

frame = pd.read_csv('data/pe32_vendors.csv.gz', dtype=str)
frame = frame.set_index('link')

av_classified = frame.count() / len(frame)
top20 = av_classified.sort_values(ascending=False)[:20]
display(top20)

TheHacker            0.999340
BitDefender          0.999259
CAT-QuickHeal        0.999239
Fortinet             0.998977
ViRobot              0.998401
Ikarus               0.998096
GData                0.997767
K7AntiVirus          0.996668
ClamAV               0.996550
F-Prot               0.996534
McAfee               0.995997
Microsoft            0.994455
nProtect             0.994321
Comodo               0.993882
Avast                0.993345
AVG                  0.992604
VBA32                0.992226
Kaspersky            0.991340
McAfee-GW-Edition    0.989802
Symantec             0.989135
dtype: float64

In [2]:
test = pd.read_csv('data/header_analyses.csv')
test = test.set_index('link')
classified = test[test.antivirus != 'n/a']
classified.dropna(inplace=True)
display(len(classified.file_type[classified.file_type.str.startswith('PE32')]))
display(len(frame))

/home/joao/virtual_envs/thesis2/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


292288

298285